## Notebook setup

In [ ]:
%pylab inline

In [ ]:
from clawpack.clawutil import nbtools
from clawpack.visclaw import animation_tools
from IPython.display import HTML
from distutils.dir_util import copy_tree
import os # If you are not using a unix distribution, find the calls to 'os' in this document and replace them\
             # with your operating system's equivalent command

In [ ]:
def show_anim(anim, form = 5):
    if form == 5:
        html_version = HTML(anim.to_html5_video())
    elif form == 'js':
        html_version = HTML(anim.to_jshtml())
    else:
        print("animation type not recognised, defaulting to HTML5 video")

    return html_version

In [ ]:
nbtools.make_exe(new=True,verbose=False)

In [ ]:
if os.path.isdir('_saved') == False:
    os.mkdir('_saved')

Function to write (any changes made by the user to) \*.data files, and then run the code from these files to create fortran data and images to be used in animation. 

In [ ]:
def compiler(k, amount_manns, out = False):
    # create *.data files from parameters in setrun.py
    rundata.write()
    # Run the code from the *.data files
    outdir,plotdir = nbtools.make_output_and_plots(label="_{}".format(k+1), verbose = out)
    print("File {} out of {} compiled".format(k + 1, amount_manns))
    return plotdir

Function to concatenate frames to be used in animation, then creating the user-facing animation. 

In [ ]:
from PIL import Image

def stitch_anim(plot_manns, amount_manns, choose_manns, exno = 1, figno = 0, form = 5):
    os.system("mkdir _plots") # LinuxOS command : makes the directory '_plots' in CLAW/apps/notebooks/geoclaw/chile2010b
    
    if exno == 1:
        frames = 5
        extra = ''
    elif exno == 2:
        frames = 19
        extra = 0
    else:
        print ("Error : Enter the exercise you are running, either 1, or 2")
        
    for a in range(exno - 1 ,frames + 1):
        im = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(plot_manns[0],a, figno))
        im_alt = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(plot_manns[1], a, figno))
        dst = Image.new('RGB', (im.width + im_alt.width, im.height))
        dst.paste(im, (0, 0))
        dst.paste(im_alt, (im.width, 0))
        dst.save('_plots/frame{:04d}fig{}.png'.format(a, figno))
    anim = animation_tools.animate_from_plotdir('_plots', figno)
    print('\nMannings n = {} on left animation, and n = {} on right '\
          'animation'.format(choose_manns[plot_manns[0]], choose_manns[plot_manns[1]]))
    return show_anim(anim, form)

Function to run and save all animations

# Get mannings

In [ ]:
# Choose purpose of current data complation
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
purpose = 'plot#or animation or both'
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
if purpose == 'plot':
    amount_manns = 2
    #vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv   
    first_mann = 0.003
    second_mann = 0.025
    #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    space_manns = abs(second_mann - first_mann)
    mid_mann = (first_mann if first_mann > second_mann else second_mann)
    
else :
    # plot family of curves around
    #vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
    mid_mann = 0.03
    amount_manns = 10
    space_manns = 0.025
    #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

manns = [None] * amount_manns
choose_manns = {}
    

for k in range (amount_manns):
    if amount_manns%2 == 0:
        manns[k] = abs(mid_mann + (k + 1 - (amount_manns)/2) * space_manns)
    else:
        manns[k] = abs(mid_mann + (k + 1 - (amount_manns + 1) /2) * space_manns)
    choose_manns[k+1] = manns[k]

for k in range (1, amount_manns + 1):
    os.system("cp setrun.py setrun_{}.py".format(k)) # LinuxOS command : copies 'setrun.py' and pastes it as renamed 'setrun_#'.py
    
print('Mannings values to be compiled : {}\n\n'.format(manns))

plot_manns = [None] * 2
if len(choose_manns) > 2:
    plot_manns[0] = int(input('Choose (1-{}) the first mannings values to'\
                          ' compare via animation: {}\n'.format(amount_manns, choose_manns)))
    plot_manns[1] = int(input('Choose (1-{}) the second mannings values to'\
                          ' compare via animation: {}\n'.format(amount_manns, choose_manns)))
else:
    plot_manns[0] = 1
    plot_manns[1] = 2

## Change variables

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
exno = 2 # If 1, can plot figure 0 or 1. If 2, can plot graphs, and has detailed timestep
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

for k in range (amount_manns):
    mod = importlib.import_module("setrun_{}".format(k+1))
    rundata = mod.setrun()
    # Variables to change
    #vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
    # Change Mannings
    rundata.geo_data.manning_coefficient = manns[k]
    # Change output times:
    rundata.clawdata.output_style = exno #This is the difference from ex1 to ex2 in the animation
    rundata.clawdata.output_times = linspace(3.5,8,19) * 3600.
    # Change gauge location:
    from clawpack.amrclaw.data import GaugeData
    rundata.replace_data("gaugedata", GaugeData())  # clear old gauge data
    rundata.gaugedata.gauges = []   # empty list
    rundata.gaugedata.gauges.append([32412, -86.392, -17.975, 0., 1.e10]) # original gauge
    rundata.gaugedata.gauges.append([123, -77.3, -12.3, 3.5*3600, 1.e10]) # relocated gauge
    #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    compiler(k, amount_manns)

# Show animation

In [ ]:
if exno == 1:
    figno = 1 # Choose either 1 or 0
else:
    figno = 1 # Do not alter
    
stitch_anim(plot_manns, amount_manns, choose_manns, exno, figno, 'js')

# Viewing gauges in Python

The file `setplot.py` can be modified to plot different things at the gauges.  We don't do that here, and instead illustrate how to directly plot things in Python.  The file `plot_gauge123.py` in this directory also implements this. So if you type

    python plot_gauge123.py
    
two png files should be produced with plots similar to those shown below.

First we read the plot data from the most recent GeoClaw run:

RUN Exercise number 2 first (you do not need to animate it though)  

In [ ]:
from setplot import setplot
plotdata = setplot()

h123 = [None] * (amount_manns)
u123 = [None] * (amount_manns)
v123 = [None] * (amount_manns)
t123 = [None] * (amount_manns)
eta123 = [None] * (amount_manns)

h32412 = [None] * (amount_manns)
u32412 = [None] * (amount_manns)
v32412 = [None] * (amount_manns)
t32412 = [None] * (amount_manns)
eta32412 = [None] * (amount_manns)


for k in range(amount_manns):
    plotdata.outdir = '_output_{}'.format(k + 1)
    # get gauge 123 data
    g123 = plotdata.getgauge(123)
    t123[k] = g123.t / 3600.  # convert to hours
    eta123[k] = g123.q[3,:]   # eta = h + B (depth plus bathymetry)
    h123[k] = g123.q[0,:]       # water depth
    u123[k] = g123.q[1,:] / h123[k]  # u = east-west velocity
    v123[k] = g123.q[2,:] / h123[k]  # v = north-south velocity
    # get gauge 32412 data
    g32412 = plotdata.getgauge(32412)
    t32412[k] = g32412.t / 3600.  # convert to hours
    eta32412[k] = g32412.q[3,:]   # eta = h + B (depth plus bathymetry)
    h32412[k] = g32412.q[0,:]       # water depth
    u32412[k] = g32412.q[1,:] / h32412[k]  # u = east-west velocity
    v32412[k] = g32412.q[2,:] / h32412[k]  # v = north-south velocity

colour = ['#fc0303', '#f59042', '#f5ce42', '#d7f542', 
          '#90f542', '#48f542', '#42f59c', '#42ddf5', 
          '#4272f5', '#8a42f5']
# for k in range(amount_manns):
#     contrast = (k*3) % 10
#     colour[k] = colour[contrast]

## plot velocity data of gauge 123

In [ ]:
subplot(1, 2, 1)
grid()
for k in range(amount_manns):
    plot(t123[k],u123[k], color = colour[k], label="Manning's n = {:.3f}".format(manns[k]))   
xlabel('Hours since earthquake')
ylabel('meters per second')
title('east-west velocities at gauge 123')

subplot(1, 2, 2)
grid()
for k in range(amount_manns):
    plot(t123[k],v123[k], color = colour[k], label="Manning's n = {:.3f}".format(manns[k]))   
legend()
title('north-south velocities at gauge 123')

tight_layout()
gcf().set_size_inches(14,6)
show()

## plot velocity of gauge 32412

In [ ]:
subplot(1, 2, 1)
grid()
for k in range(amount_manns):
    plot(t32412[k],u32412[k], color = colour[k], label="Manning's n = {:.3f}".format(manns[k]))   
xlabel('Hours since earthquake')
ylabel('meters per second')
title('east-west velocities at gauge 32412')

subplot(1, 2, 2)
grid()
for k in range(amount_manns):
    plot(t32412[k],v32412[k], color = colour[k], label="Manning's n = {:.3f}".format(manns[k]))   
legend()
title('north-south velocities at gauge 32412')

tight_layout()
gcf().set_size_inches(14,6)
show()

## plot height of gauge 123

In [ ]:
grid()
for k in range(amount_manns):
    plot(t123[k],eta123[k], color = colour[k], label="Manning's n = {:.3f}".format(manns[k]))
legend()
xlim(3,8)
xlabel('Hours since earthquake')
ylabel('meters')
title('Sea surface elevation at gauge 123')

gcf().set_size_inches(14,6)
show()

## plot height of gauge 32412 against real life data

In [ ]:
ax = gca()
 
ax.set_prop_cycle(color=colour) 

xmin, xmax, ymin, ymax = (3, 7, -0.20, 0.30)

for k in range(amount_manns):
    ax.plot(t32412[k],eta32412[k], label="Manning's n = {:.3f}".format(manns[k]))
ax.plot(0,0,color='k', label="Real life data")
ax.legend()
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xlabel('Hours since earthquake')
ax.set_ylabel('Meters')
ax.set_title('Sea surface elevation at gauge 32412')

ax.set_zorder(2)
ax.set_facecolor('none')

ax_tw_x = ax.twinx()
ax_tw_x.axis('off')
ax2 = ax_tw_x.twiny()

im = imread('gauge32412-actual-detide.png')
ax2.imshow(im, extent=[xmin, xmax, ymin, ymax], aspect='auto')
ax2.axis('off')
gcf().set_size_inches(14,6)
show()

# TESTING AREA

Notes :
- Save plots and animations (can I use the anim. in the first function? like anim.save?)
- Write figure legends
- Use G123 data to test different values given in the paper for areas such as forests,
    importance of mangrove trees was undervalued
- colours need to have more distinction

In [ ]:
# from PIL import Image

# def stitch_save_anim(choose_manns, plot_manns, exno = 1, figno = 0):
#     os.system("mkdir _plots")
    
#     if exno == 1:
#         frames = 5
#         extra = ''
#     elif exno == 2:
#         frames = 19
#         extra = 0
#     else:
#         print ("Error : Enter the exercise you are running, either 1, or 2")
        
#     for a in range(exno - 1 ,frames + 1):
#         im = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(plot_manns[0],a, figno))
#         im_alt = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(plot_manns[1], a, figno))
#         dst = Image.new('RGB', (im.width + im_alt.width, im.height))
#         dst.paste(im, (0, 0))
#         dst.paste(im_alt, (im.width, 0))
#         dst.save('_plots/frame{:04d}fig{}.png'.format(a, figno))
#         source = '_plots/'
#         dest = '_saved/mann_{}_{}-ex{}-fig{}'.format(choose_manns[plot_manns[0]]\
#                                                      ,choose_manns[plot_manns[1]],1,0)
#         copy_tree(source, dest)
        
# #         animation_tools.animate_from_plotdir('_plots', figno)
# #     print('\nMannings n = {} on left animation, and n = {} on right '\
# #           'animation'.format(choose_manns[plot_manns[0]], choose_manns[plot_manns[1]]))
# #     return show_anim(anim)
                  
# for j in range (3):
#     for k in range (amount_manns):
#         mod = importlib.import_module("setrun_{}".format(k+1))
#         rundata = mod.setrun()
#         # Variables to change
#         #vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
#         # Change Mannings
#         rundata.geo_data.manning_coefficient = manns[k]
#         # Change output times:
#         if j == 2:
#             rundata.clawdata.output_style = 2 #This is the difference from ex1 to ex2 in the animation
#             rundata.clawdata.output_times = linspace(3.5,8,19) * 3600.
#             # Change gauge location:
#             from clawpack.amrclaw.data import GaugeData
#             rundata.replace_data("gaugedata", GaugeData())  # clear old gauge data
#             rundata.gaugedata.gauges = []   # empty list
#             rundata.gaugedata.gauges.append([32412, -86.392, -17.975, 0., 1.e10]) # original gauge
#             rundata.gaugedata.gauges.append([123, -77.3, -12.3, 3.5*3600, 1.e10]) # relocated gauge
#         #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#         compiler(k, amount_manns)
#     if j < 2:
#         stitch_save_anim(choose_manns, plot_manns,1,j)
#     else:
#         stitch_save_anim(choose_manns, plot_manns,2,1)